# Урок 3. Системы управления базами данных MongoDB и SQLite в Python
1) Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД (без датафрейма)

2) Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы. Поиск по двум полям (мин и макс зарплату)

3) Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта

In [2]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from pprint import pprint
import re

In [358]:
def preporation_parser(vacancy):
    vacancy_date = []
    
    params = {
        'text': vacancy, 
        'area': "1438",
        'search_field': 'name', 
        'page': ''
    }
    
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36'
    }
    
    link = 'https://hh.ru/search/vacancy' 
    html = requests.get(link, params=params, headers=headers)
    
    if html.ok:
        parsed_html = bs(html.text,'html.parser')
            
        vacancy_items = parsed_html.find('div', {'data-qa': 'vacancy-serp__results'}) \
                                        .find_all('div', {'class': 'vacancy-serp-item'})
        
        pager_next = parsed_html.find_all('a', {'class':'bloko-button HH-Pager-Controls-Next HH-Pager-Control'})
        if not pager_next:
            last_page = 1
        else:
            last_page = int(parsed_html.find_all('a', {'class':'HH-Pager-Control'})[-2].getText())
        
    for page in range(0, last_page):
        params['page'] = page
        html = requests.get(link, params=params, headers=headers)
        
        if html.ok:
            parsed_html = bs(html.text,'html.parser')
            
            vacancy_items = parsed_html.find('div', {'data-qa': 'vacancy-serp__results'}) \
                                        .find_all('div', {'class': 'vacancy-serp-item'})
                
            for item in vacancy_items:
                vacancy_date.append(parser_hh(item))
    #pprint(vacancy_date)            
    return vacancy_date

In [362]:
def parser_hh(item):
    vacancy_date = {}
    
    # vacancy name
    vacancy_name = item.find('div',{'class':'vacancy-serp-item__info'}).getText().replace(u'\xa0', u' ')
    vacancy_date['name'] = vacancy_name
    
    # company_name
    company_name = item.find('div',{'class':'vacancy-serp-item__meta-info'}).find('a').getText()
    vacancy_date['company_name'] = company_name
    
    # city 
    city = item.find('span',{'data-qa':'vacancy-serp__vacancy-address'}).getText()
    vacancy_date['city'] = city
    
    # salary
    
    salary = item.find('div',{'class':'vacancy-serp-item__sidebar'})
    if not salary:
        salary_min = None
        salary_max = None
        salary_currency = None
    else:
        salary = salary.getText() \
                        .replace(u'\xa0', u'')
        
        salary = re.split(r'\s|-', salary)
        
        if salary[0] == 'до':
            salary_min = None
            salary_max = int(salary[1])
            salary_currency = salary[2]
        elif salary[0] == 'от':
            salary_min = int(salary[1])
            salary_max = None
            salary_currency = salary[2]
        elif salary[0] == '':
            salary_min = None
            salary_max = None
            salary_currency = None
            salary_currency = None
        else:
            salary_min = int(salary[0])
            salary_max = int(salary[1])
            salary_currency = salary[2]
        
        
        
    vacancy_date['salary_min'] = salary_min
    vacancy_date['salary_max'] = salary_max
    vacancy_date['salary_currency'] = salary_currency
    
    # link
    vacancy_link = item.find('a',{'class':'bloko-link'})['href']
    vacancy_date['link'] = vacancy_link
    
    # id 
    #vacancy_id = int(re.findall(r'\d+', vacancy_link)[0])
    #vacancy_date['_id'] = vacancy_id
    
    # site
    vacancy_date['site'] = 'hh.ru'
    
    
    return vacancy_date

In [363]:
from pymongo import MongoClient
from pymongo.errors import BulkWriteError

In [364]:
job = input('Вакансия: ')
#salary = input('Зарплата: ')

Вакансия: программист


In [365]:
# клиент для подключения к серверу
client = MongoClient('localhost', 27017)

In [366]:
db = client['my_database']

Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД (без датафрейма)

In [367]:
vacancys_data = preporation_parser(job)

In [368]:
#pprint(vacancys_data)

In [369]:
def write_vacancys(vacancys_date):
    # создаю коллекцию вакансий
    vacancys = db.vacancys
    
    #vacancys.drop()
    #vacancys.insert_many(vacancys_date)
    #vacancys.delete_many({})
     
    #return vacancys
    for vacancy in vacancys.find({}):
        pprint(vacancy)

In [370]:
write_vacancys(vacancys_data)

{'_id': 37754198,
 'city': 'Краснодар',
 'company_name': 'ООО Метротел',
 'link': 'https://krasnodar.hh.ru/vacancy/37754198?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'name': 'PHP-разработчик',
 'salary_currency': 'руб.',
 'salary_max': None,
 'salary_min': 120000,
 'site': 'hh.ru'}
{'_id': 37443536,
 'city': 'Краснодар',
 'company_name': ' Мазалов Дмитрий Александрович',
 'link': 'https://krasnodar.hh.ru/vacancy/37443536?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'name': 'Программист',
 'salary_currency': None,
 'salary_max': None,
 'salary_min': None,
 'site': 'hh.ru'}
{'_id': 37789459,
 'city': 'Краснодар',
 'company_name': ' Работа-это проСТО',
 'link': 'https://krasnodar.hh.ru/vacancy/37789459?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'name': 'Программист C# / Разработчик С#',
 'salary_currency': 'руб.',
 'salary_max': 120000,
 'salary_min': 120000,
 'site': 'hh.ru'}
{'_id': 3760910

2) Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы. Поиск по двум полям (мин и макс зарплату)

In [45]:
def find_vacancy_by_salary(salary, vacancys_data):
    ''' Функция вывода вакансий с определенной заработной платой
        Если зарплата больше той, что ввел пользователь,
        то выведем такую вакансию.'''
    
    write_vacancys(vacancys_data)
    vacancys_salary_more = vacancys.find({'$or':[{'salary_min':{'$gt':salary}}, {'salary_max':{'$gt':salary}}]})
    
    return vacancys_salary_more
    

In [46]:
find_vacancy_by_salary(salary, vacancys_data)

{'_id': ObjectId('5efed56ac4bfbb6ef57a351c'),
 'city': 'Москва',
 'company_name': ' CATAPULTO.RU',
 'link': 'https://krasnodar.hh.ru/vacancy/37674249?query=python',
 'name': 'Программист-разработчик python (midle)',
 'salary_currency': 'руб.',
 'salary_max': '180000',
 'salary_min': '100000',
 'site': 'hh.ru'}
{'_id': ObjectId('5efed56ac4bfbb6ef57a351d'),
 'city': 'Санкт-Петербург, Петроградская',
 'company_name': ' Интеллектуальные системы',
 'link': 'https://krasnodar.hh.ru/vacancy/37765447?query=python',
 'name': 'Скилловый Python Developer на бэкенд',
 'salary_currency': 'руб.',
 'salary_max': None,
 'salary_min': '250000',
 'site': 'hh.ru'}
{'_id': ObjectId('5efed56ac4bfbb6ef57a351e'),
 'city': 'Москва',
 'company_name': 'ООО Чистый Маркетинг',
 'link': 'https://krasnodar.hh.ru/vacancy/37461514?query=python',
 'name': 'Python / Django fullstack программист',
 'salary_currency': 'руб.',
 'salary_max': '180000',
 'salary_min': '110000',
 'site': 'hh.ru'}
{'_id': ObjectId('5efed56ac4

NameError: name 'vacancys' is not defined

3) Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта

In [383]:
def new_vacations(vacancys_data):
    for data in vacancys_data:
        return db.vacancys.update(data, {"upsert":True})
    

In [407]:
new_vacations(data)

/Users/gleb/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  


{'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}